In [1]:
from create_and_run_slurmies import create_eval_job_df, create_slurmies
import argparse

%cd ~/openfold

/net/pulsar/home/koes/jok120/openfold


In [2]:
# make a fake args object with exp_directory, eval_job_csv, slurmie_outdir, base_slurm_str, location
args = argparse.Namespace()
args.exp_directory = "/net/pulsar/home/koes/jok120/openfold/jk_research/evaluations/experiment_directory.csv"
args.eval_job_csv = "/net/pulsar/home/koes/jok120/openfold/jk_research/evaluations/230418/eval_jobs.csv"
args.slurmie_outdir = "out/evaluations/230418-auto/slurmies"
args.base_slurm_str = "/net/pulsar/home/koes/jok120/openfold/jk_research/evaluations/230418/subjob_skeleton.slurm"
args.location = "g019"

In [3]:
df = create_eval_job_df(args.exp_directory, args.eval_job_csv, args.location)

In [4]:
df

,exp_suffix,exp_name,eval_me,wandb_id,location,notes,new_exp_name,exp_dir,checkpoint_path
1,eval_s945,baseline-scnmin-noomm-00,True,3lzb4s3o,g019,Our baseline on scnmin data. Overfits the data.,baseline_scnmin_noomm_00_eval_s945,out/experiments/230319/finetune-openfold-02/3l...,out/experiments/230319/finetune-openfold-02/3l...
2,eval_sx,baseline-scnmin-noomm-00,True,3lzb4s3o,g019,Our baseline on scnmin data. Overfits the data.,baseline_scnmin_noomm_00_eval_sx,out/experiments/230319/finetune-openfold-02/3l...,out/experiments/230319/finetune-openfold-02/3l...
4,eval_sx,xfer-scnmin-omm-01,True,c2bs3y0h,g019,Resume baseline-scnmin-noomm-00 at about 1k st...,xfer_scnmin_omm_01_eval_sx,out/experiments/230407_xfer01/finetune-openfol...,out/experiments/230407_xfer01/finetune-openfol...


In [5]:
from experiments import Experiment, TrainingExperiment, EvaluationExperiment

# Create EvaluationExperiment objects from the dataframe
eval_exps = []
for i, row in df.iterrows():
    eval_exp = EvaluationExperiment(
        exp_name=row["new_exp_name"],
        wandb_id=row["wandb_id"],
        location=row["location"],
        notes=f"Evaluating {row['exp_name']}",
        exp_suffix=row["exp_suffix"],
        checkpoint_path=row["checkpoint_path"],
    )
    eval_exps.append(eval_exp)

In [8]:
print(eval_exps[0].get_slurm_script_str("job1"))

#!/bin/bash
#SBATCH --job-name=job1
#SBATCH --nodes=1
#SBATCH --gres=gpu:1
#SBATCH --ntasks-per-node=4
#SBATCH --partition=dept_gpu
#SBATCH --time=6-00:00:00
#SBATCH --output="/net/pulsar/home/koes/jok120/openfold/out/%A_%6a.out"
#SBATCH --mail-type=ALL
#SBATCH --mail-user=jok120@pitt.edu
#SBATCH --nodelist=g019


############################
##       Description      ##
############################
echo "Running job ${SLURM_JOB_ID} with ${SLURM_NTASKS} workers on node ${SLURMD_NODENAME}."
echo "Experiment: baseline_scnmin_noomm_00_eval_s945"

############################
##       Environment      ##
############################
source scripts/activate_conda_env.sh
module load cuda/11.3.0
module load gcc/8.2.0


############################
##     Array Job Exec.    ##
############################
mkdir -p out/evaluations/baseline_scnmin_noomm_00_eval_s945


./train_openfold.py data/train_structs/scnmin_structs/ data/alignments/scnmin_alignments/ data/template_structures/roda_pdbs_snap

In [7]:
%pwd

'/net/pulsar/home/koes/jok120/openfold'